# MLSS2019: Bayesian Deep Learning

In this tutorial we will learn what basic building blocks are needed
to endow (deep) neural networks with uncertainty estimates, and how
this can be used in active learning or expert-in-the-loop pipelines.

The plan of the tutorial
1. [Setup and imports](#Setup-and-imports)
2. [Bayesian Active Learning with images](#Bayesian-Active-Learning-with-images)
   1. [the Acquisition Function](#the-Acquisition-Function)
   2. [Actively learning MNIST](#Actively-Learning-MNIST)
3. 

**(note)**
* to view documentation on something  type in `something?` (with one question mark)
* to view code of something type in `something??` (with two question marks).

<br>

## Setup and imports

In this section we import necessary modules and functions and
define the computational device.

First, we install some boilerplate service code for this tutorial.

In [ ]:
!pip install -q --upgrade git+https://github.com/ivannz/mlss2019-bayesian-deep-learning.git@alternative

Next, numpy for computing, matplotlib for plotting and tqdm for progress bars.

In [ ]:
import tqdm
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

For deep learning stuff will be using [pytorch](https://pytorch.org/).

If you are unfamiliar with it, it is basically like `numpy` with autograd,
native GPU support, and tools for building training and serializing models.
<!-- (and with `axis` argument replaced with `dim` :) -->

There are good introductory tutorials on `pytorch`, like this
[one](https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html).

In [ ]:
import torch
import torch.nn.functional as F

device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")

We will need some functionality from scikit

In [ ]:
from sklearn.metrics import confusion_matrix

Next we import the boilerplate code.

* a procedure that implements a minibatch SGD **fit** loop
* a function, that **evaluates** the model on the provided dataset

In [ ]:
from mlss2019bdl import fit, predict

The algorithm to sample a random function is:
* for $b = 1... B$ do:

  1. draw an independent realization $f_b\colon \mathcal{X} \to \mathcal{Y}$
  with from the process $\{f_\omega\}_{\omega \sim q(\omega)}$
  2. get $\hat{y}_{bi} = f_b(\tilde{x}_i)$ for $i=1 .. m$


In [ ]:
from mlss2019bdl.bdl import freeze, unfreeze

def sample_function(model, dataset, n_samples=1, verbose=False):
    """Draw a realization of a random function."""
    outputs = []
    for _ in tqdm.tqdm(range(n_samples), disable=not verbose):
        freeze(model)

        outputs.append(predict(model, dataset))

    unfreeze(model)

    return torch.stack(outputs, dim=0)

<br>

## Bayesian Active Learning with images

* Data labelling is costly and time consuming
* unlabeled instances are essentially free

**Goal** Achieve high performance with fewer labels by
identifying the best instances to learn from

Essential blocks of active learning:

* a **model** $m$ capable of quantifying uncertainty (preferably a Bayesian model)
* an **acquisition function** $a\colon \mathcal{M} \times \mathcal{X}^* \to \mathbb{R}$
  that for any finite set of inputs $S\subset \mathcal{X}$ quantifies their usefulness
  to the model $m\in \mathcal{M}$
* a labelling **oracle**, e.g. a human expert

The main loop of active learning:

1. fit $m$ on $\mathcal{S}_{\mathrm{labelled}}$

2. get exact (or approximate) $$
    \mathcal{S}^* \in \arg \max\limits_{S \subseteq \mathcal{S}_\mathrm{unlabelled}}
        a(m, S)
$$ satisfying **budget constraints** and **without** access to targets
(constraints, like $\lvert S \rvert \leq \ell$ or other economically motivated ones).

3. request the **oracle** to provide labels for each $x\in \mathcal{S}^*$

4. update $
\mathcal{S}_{\mathrm{labelled}}
    \leftarrow \mathcal{S}^*
        \cup \mathcal{S}_{\mathrm{labelled}}
$ and goto 1.

We already have a Bayesian model that can be used to reason
about uncertainty, so let's focus on the acquisition function.

<br>

### the Acquisition Function

There are many acquisition criteria (borrowed from [Gal17a](http://proceedings.mlr.press/v70/gal17a.html)):
* Classification
  * Max entropy (plain uncertainty)
  * Maximal information about parameters and predictions (mutual information)
  * Variance ratios
  * Mean standard deviation
  * **BALD**
* Regression
  * predictive variance

**BALD** (Bayesian Active Learning by Disagreement) acquisition
criterion is based on the posterior mutual information between model's
predictions $y_x$ at some point $x$ and model's parameters $\omega$:

$$\begin{align}
    a(m, S)
        &= \sum_{x\in S} a(m, \{x\})
        \\
    a(m, \{x\})
        &= \mathbb{I}(y_x; \omega \mid x, m, D)
\end{align}
    \,, \tag{bald} $$

with the [**Mutual Information**](https://en.wikipedia.org/wiki/Mutual_information#Relation_to_Kullback%E2%80%93Leibler_divergence)
(**MI**)
$$
    \mathbb{I}(y_x; \omega \mid x, m, D)
        = \mathbb{H}\bigl(
            \mathbb{E}_{\omega \sim q(\omega\mid m, D)}
                p(y_x \,\mid\, x, \omega, m, D)
        \bigr)
        - \mathbb{E}_{\omega \sim q(\omega\mid m, D)}
            \mathbb{H}\bigl(
                p(y_x \,\mid\, x, \omega, m, D)
            \bigr)
    \,, \tag{mi} $$

and the [(differential) **entropy**](https://en.wikipedia.org/wiki/Differential_entropy#Differential_entropies_for_various_distributions)
(all densities and/or probability mass functions can be conditional):

$$
    \mathbb{H}(p(y))
        = - \mathbb{E}_{y\sim p} \log p(y)
    \,. $$

Instead of the exact formula for **MI** we shall use its **Monte Carlo** (**MC**)
approximation, since the expectations are analytically or numerically
tractable only in simple low dimensional cases.
<!-- probability integrals are still integrals -->

Consider an iid sample $\mathcal{W} = (\omega_b)_{b=1}^B \sim q(\omega \mid m, D)$
of size $B$. The **MC** approximation of the mutual information is

$$
    \mathbb{I}_\mathrm{MC}(y_x; \omega \mid x, m, D)
        = \mathbb{H}\bigl(
            \hat{\mathbb{E}}_{\omega \sim \mathcal{W}}
                p(y_x \,\mid\, x, \omega, m, D)
        \bigr)
        - \hat{\mathbb{E}}_{\omega \sim \mathcal{W}}
            \mathbb{H}\bigl(
                p(y_x \,\mid\, x, \omega, m, D)
            \bigr)
    \,, \tag{mi-mc} $$

where $\hat{\mathbb{E}}_{\omega \sim \mathcal{W}} h(\omega) = \tfrac1B \sum_j h(\omega_j)$
denotes the expectation with respect to the empirical probability measure induced
by the sample $\mathcal{W}$.

<br>

#### (task) implementing entropy

For categorical (discrete) random variables $y \sim \mathcal{Cat}(\mathbf{p})$,
$\mathbf{p} \in \{ \mu \in [0, 1]^d \colon \sum_k \mu_k = 1\}$, the entropy is

$$
    \mathbb{H}(p(y))
        = - \mathbb{E}_{y\sim p(y)} \log p(y)
        = - \sum_k p_k \log p_k
    \,. $$

**(note)** although in calculus $0 \cdot \log 0 = 0$ (because
$\lim_{p\downarrow 0} p \cdot \log p = 0$), in floating point
arithmetic $0 \cdot \log 0 = \mathrm{NaN}$. So you need to add
some **really tiny float number** to the argument of $\log$.

In [ ]:
def entropy(proba):
    """Compute the entropy along the last dimension."""

    ## Exercise: get the entropy of a tensor with distributions
    #  along the last axis.

    return - torch.kl_div(torch.tensor(0.).to(proba), proba).sum(dim=-1)
    return - torch.sum(proba * torch.log(proba + 1e-20), dim=-1)

    pass

<br>

#### (task) implementing mutual information

Consider a tensor $p_{bik}$ of probabilities $p(y_{x_i}=k \mid x_i, \omega_b, m, D)$
with $\omega_b \sim q(\omega \mid m, D)$.

Let's implement a procedure that computes the **MC** estimate 
of the posterior predictive distribution

$$
\hat{p}(y_x\mid x, m, D)
    = \hat{\mathbb{E}}_{\omega \sim \mathcal{W}}
        \,p(y_x \mid x, \omega, m, D)
    \,, $$

its **entropy** $
    \mathbb{H}\bigl(\hat{p}(y\mid x, m, D)\bigr)
$ and **mutual information** $
    \mathbb{I}_\mathrm{MC}(y_x ; \omega\mid x, m, D)
$

In [ ]:
def mutual_information(proba):
    ## Exercise: compute a Monte Carlo estimator of the predictive
    ##   distribution, its entropy and MI `H E_w p(., w) - E_w H p(., w)`

    proba_avg = proba.mean(dim=0)

    entropy_expected = entropy(proba_avg)
    expected_entropy = entropy(proba).mean(dim=0)

    mut_info = entropy_expected - expected_entropy

    pass

    return proba_avg, entropy_expected, mut_info

<br>

#### (task) implementing BALD acqustion

The acquisition function that we will implement takes in the
sample mutual information and returns the indices of selected
points.



Note that $a(m, S)$ is additively separable, i.e. equals $\sum_{x\in S} a(m, \{x\})$.
This implies that

$$
\begin{align}
    \max_{S \subseteq \mathcal{S}_\mathrm{unlabelled}} a(m, S)
        &= \max_{z \in \mathcal{S}_\mathrm{unlabelled}}
            \max_{F \in \mathcal{S}_\mathrm{unlabelled} \setminus \{z\}}
            \sum_{x\in F \cup \{x\}} a(m, \{x\})
        \\
        &= \max_{z \in \mathcal{S}_\mathrm{unlabelled}}
            a(m, \{z\})
            + \max_{F \in \mathcal{S}_\mathrm{unlabelled} \setminus \{z\}}
                \sum_{x\in F} a(m, \{x\})
\end{align}
    \,. $$

Therefore selecting the $\ell$ `most interesting` points from $\mathcal{S}_\mathrm{unlabelled}$
is trivial.


In [ ]:
def acq_bald(mutual_info, n_points=10):
    ## Exercise: implement the acquisition

    indices = mutual_info.argsort()

    return indices[-n_points:]

    pass

<br>

**(note)** A drawback of the `pointwise` top-$\ell$ procedure above is
that, although it acquires individually informative instances, altogether
they might end up **being** `jointly poorly informative`. This can be
corrected if we would seek the highest mutual information among finite
sets $S \subseteq \mathcal{S}_\mathrm{unlabelled}$ of size $\ell$.
Such acquisition function is called **batch-BALD**
([Kirsch et al.; 2019](https://arxiv.org/abs/1906.08158.pdf)):

$$\begin{align}
    a(m, S)
        &= \mathbb{I}\bigl((y_x)_{x\in S}; \omega \mid S, m \bigr)
        = \mathbb{H} \bigl(
            \mathbb{E}_{\omega \sim q(\omega\mid m)} p\bigl((y_x)_{x\in S}\mid S, \omega, m \bigr)
        \bigr)
        - \mathbb{E}_{\omega \sim q(\omega\mid m)} H\bigl(
            p\bigl((y_x)_{x\in S}\mid S, \omega, m \bigr)
        \bigr)
\end{align}
    \,. \tag{batch-bald} $$

This criterion requires exponentially large number of computations and
memory, however there are working solutions like random sampling of subsets
$\mathcal{S}$ of size $\ell$ from $\mathcal{S}_\mathrm{unlabelled}$ or
greedy maximization of this *submodular* criterion.

> Kirsch, A., van Amersfoort, J., & Gal, Y. (2019). BatchBALD: Efficient and Diverse Batch Acquisition for Deep Bayesian Active Learning. arXiv preprint [arXiv:1906.08158](https://arxiv.org/abs/1906.08158.pdf)

<br>

#### (task*) Unbiased estimator of entropy and mutual information

The first term in the **MC** estimate of the mutual information is the
so-called **plug-in** estimator of the entropy:

$$
    \hat{H}
        = \mathbb{H}(\hat{p}) = - \sum_k \hat{p}_k \log \hat{p}_k
    \,, $$

where $\hat{p}_k = \tfrac1B \sum_b p_{bk}$ is the full sample estimator
of the probabilities.

It is known that this plug-in estimate is biased
(see [blog: Nowozin, 2015](http://www.nowozin.net/sebastian/blog/estimating-discrete-entropy-part-1.html)
and references therein, also this [notebook](https://colab.research.google.com/drive/1z9ZDNM6NFmuFnU28d8UO0Qymbd2LiNJW)). <!--($\log$ + Jensen)-->
In order to correct for small-sample bias we can use
[jackknife resampling](https://en.wikipedia.org/wiki/Jackknife_resampling).
It derives an estimate of the finite sample bias from the leave-one-out
estimators of the entropy and is relatively computationally cheap
(see [blog: Nowozin, 2015](http://www.nowozin.net/sebastian/blog/estimating-discrete-entropy-part-2.html),
[Miller, R. G. (1974)](http://www.math.ntu.edu.tw/~hchen/teaching/LargeSample/references/Miller74jackknife.pdf) and these [notes](http://people.bu.edu/aimcinto/jackknife.pdf)).

The jackknife correction of a plug-in estimator $\mathbb{H}(\cdot)$
is computed thus: given a sample $(p_b)_{b=1}^B$ with $p_b$ -- discrete distribution on $1..K$
* for each $b=1.. B$
  * get the leave-one-out estimator: $\hat{p}_k^{-b} = \tfrac1{B-1} \sum_{j\neq b} p_{jk}$
  * compute the plug-in entropy estimator: $\hat{H}_{-b} = \mathbb{H}(\hat{p}^{-b})$
* then compute the bias-corrected entropy estimator $
\hat{H}_J
    = \hat{H} + (B - 1) \bigl\{
        \hat{H} - \tfrac1B \sum_b \hat{H}^{-b}
    \bigr\}
$

**(note)** when we knock the $i$-th data point out of the sample mean
$\mu = \tfrac1n \sum_i x_i$ and recompute the mean $\mu_{-i}$ we get
the following relation
$$ \mu_{-i}
    = \frac1{n-1} \sum_{j\neq i} x_j
    = \frac{n}{n-1} \mu - \tfrac1{n-1} x_i
    = \mu + \frac{\mu - x_i}{n-1}
    \,. $$
This makes it possible to quickly compute leave-one-out estimators of
discrete probability distribution.

In [ ]:
if True:
    def mutual_information(proba):
        ## Exercise: MC estimate of the predictive distribution, entropy and MI
        ##  mutual information `H E_w p(., w) - E_w H p(., w)` with jackknife
        ##  correction.

        proba_avg = proba.mean(dim=0)

        # plug-in estimate of entropy
        entropy_expected = entropy(proba_avg)

        # jackknife correction
        proba_loo = proba_avg + (proba_avg - proba) / (len(proba) - 1)

        expected_entropy_loo = entropy(proba_loo).mean(dim=0)
        entropy_expected += (len(proba) - 1) * (entropy_expected - expected_entropy_loo)

        # expected entropy is unbiased
        expected_entropy = entropy(proba).mean(dim=0)

        mut_info = entropy_expected - expected_entropy

        pass

        return proba_avg, entropy_expected, mut_info

<br>

### Actively Learning MNIST

We will partially replicate figure 1. in [Gat et al. (2017): p. 4](http://proceedings.mlr.press/v70/gal17a.html),

> Gal, Y., Islam, R. & Ghahramani, Z.. (2017). Deep Bayesian Active Learning with Image Data. Proceedings of the 34th International Conference on Machine Learning, in [PMLR 70:1183-1192](http://proceedings.mlr.press/v70/gal17a.html)


Prepare the datasets from the `train` part of [MNIST](http://yann.lecun.com/exdb/mnist/):
* ($\mathcal{S}_\mathrm{train}$) initial **training**:
  **empty** -- learn from scratch
  <strike>21 images, purposefully highly imbalanced classes (even absent ones)</strike>
* ($\mathcal{S}_\mathrm{valid}$) our **validation**:
  $5000$ images, stratified
* ($\mathcal{S}_\mathrm{pool}$) acquisition **pool**:
  all remaining images

The true test sample of MNIST is in $\mathcal{S}_\mathrm{test}$ -- we
will use it to evaluate the final performance.

In [ ]:
from mlss2019bdl.dataset import get_dataset

S_train, S_pool, S_valid, S_test = get_dataset(
    n_train=0, n_valid=5000, name="MNIST",
    random_state=722_257_201, path="./data")

**(note)** We may just as well use [Kuzushiji-MNIST](https://github.com/rois-codh/kmnist)

Now, a function to plot images in a small dataset. 

In [ ]:
from mlss2019bdl.flex import plot

def display(dataset, title=None, show_balance=True, figsize=None):
    images, targets = dataset.tensors
    if not show_balance:
        balance = ""

    else:
        body = [f"{n:2d}" if n > 0 else " *"
                for n in label_counts(targets)]
        balance = "(freq) [ " + ' '.join(body) + " ]"

    # a canvas
    fig, ax = plt.subplots(1, 1, figsize=figsize)

    # show the images
    plot(ax, images, cmap=plt.cm.bone)

    # produce a title
    title = "" if title is None else title
    title = title + (" " if title else "")
    ax.set_title(f"{title}{balance}")

    plt.show()
    plt.close()


def label_counts(labels, n_labels=10):
    return np.bincount(labels.numpy(), minlength=10)

In [ ]:
display(S_train, title="Train")

<br>

#### Necessary components for the loop

We need to be able to manipulate the datasets for the **main active learning**
loop. We begin by implementing the following primitives:
* `take` collect the instances at the specified indices into a **new dataset** (object)
* `append` add one dataset to another
* `delete` drops the instances at the specified locations form the copy of the **dataset**

In [ ]:
from mlss2019bdl.dataset import take, delete, append

For the **main active learning** loop, besides manipulating the datasets,
we shall also need a function to **predict and acquire** and evaluate
holdout **performance**.


In [ ]:
def predict_proba(model, dataset, n_samples=1):
    logits = sample_function(model, dataset, n_samples=n_samples)

    # logit-scores should be transformed into a proper distribution
    return F.softmax(logits, dim=-1)

In [ ]:
def acquire(model, dataset, n_points=10, n_samples=1):
    proba = predict_proba(model, dataset, n_samples=n_samples)

    _, _, mutual_info = mutual_information(proba)

    return acq_bald(mutual_info, n_points)

In [ ]:
def evaluate(model, dataset, n_samples=1):
    proba = predict_proba(model, dataset, n_samples=n_samples)
    
    proba_avg = proba.mean(dim=0)

    predicted = proba_avg.argmax(dim=-1).numpy()
    target = dataset.tensors[1].cpu().numpy()

    return confusion_matrix(target, predicted)

<br>

Much like the `SimpleModel` class above in $1$d section,
let's implement a simple deep convolutional network.

In [ ]:
from torch.nn import Linear, Conv2d

from mlss2019bdl.bdl import DropoutLinear, DropoutConv2d

class CNNModel(torch.nn.Module):
    def __init__(self, p=0.5):
        super().__init__()

        self.conv_block = torch.nn.Sequential(
            Conv2d(1, 20, 5, 1),
            torch.nn.ReLU(),
            torch.nn.AvgPool2d(2),

            DropoutConv2d(20, 50, 5, 1, p=p),
            torch.nn.ReLU(),
            torch.nn.AvgPool2d(2)
        )

        self.fc1 = DropoutLinear(4 * 4 * 50, 400, p=p)
        self.out = DropoutLinear(400, 10, p=p)

    def forward(self, input):
        """Take images and compute their class logits."""
        x = self.conv_block(input).flatten(1)

        return self.out(F.relu(self.fc1(x)))

In [ ]:
model = CNNModel(p=0.5)

model.to(device)

<br>

To fit the classifier that outputs raw logit scores, we typically
normalize outputs via `F.log_softmax` and then feed into them into
`F.nll_loss`, which computes the negative $\log$-likelihood of a
categorical distribution.

However, this is less numerically stable then using `F.cross_entropy`,
which is essentially` log_softmax + nll` fused into one stable operation.
It is good practice to pay attention to numerical stability, especially
when working with `float32`.

In [ ]:
# from mlss2019bdl.bdl import penalties

def cross_entropy(model, X, y):
    return F.cross_entropy(model(X), y)  # + sum(penalties(model)) * 1e-4

<br>

#### (task) Implementing the active learning step

Let's code the core of the active learning loop:

1. fit on **train**, then (optional) evaluate on **holdout**
2. acquire from **pool**
3. add to **train** (removing from **pool**)


In [ ]:
def fit_acquire(model, S_train, S_pool,
                         n_epochs=5, n_points=10, n_samples=11):
    ## Exercise: implement the fit-acquire loop

    # 1. fit on S_train using `cross_entropy`, set `weight_decay` to 1e-4
    fit(model, S_train, criterion=cross_entropy, n_epochs=n_epochs, weight_decay=1e-4)

    # 2. acquire new instances from S_pool
    indices = acquire(model, S_pool, n_points=n_points, n_samples=n_samples)

    # 3. query the pool for the chosen instances, then take-append-delete
    S_requested = take(S_pool, indices)
    S_train = append(S_train, S_requested)
    S_pool = delete(S_pool, indices)

    pass

    return model, S_train, S_pool, S_requested

<br>

Let's run the fit-acquire procedure in a loop:

In [ ]:
n_epochs, n_samples = 5, 11
n_active, n_points = 75, 10

display(S_train, title="initial train")

scores = []
balances = [label_counts(S_train.tensors[1])]
for step in range(n_active):

    model, S_train, S_pool, S_requested = fit_acquire(
        model, S_train, S_pool, n_epochs=n_epochs,
        n_points=n_points, n_samples=n_samples)

    # (optional) track validation score
    score_matrix = evaluate(model, S_valid, n_samples=n_samples)

    # (optional) report accuracy and the statistics on the acquired batch
    balances.append(label_counts(S_train.tensors[1]))
    scores.append(score_matrix)

    accuracy = score_matrix.diagonal().sum() / score_matrix.sum()
    display(S_requested, title=f"# {len(S_train)} (Acc.) {accuracy:.1%}")

Train of the final $\mathcal{S}_\mathrm{train}$ and evaluate the result

In [ ]:
balances = np.stack(balances, axis=0)

fit(model, S_train, criterion=cross_entropy, n_epochs=n_epochs, weight_decay=1e-4)
scores.append(evaluate(model, S_valid, n_samples=n_samples))

scores = np.stack(scores, axis=0)

display(S_train, title="final train", figsize=(16, 9))

<br>

### Results

Let's see the dynamics of the frequency of each class in $\mathcal{S}_\mathrm{train}$

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 7))

lines = ax.plot(balances, lw=2)
plt.legend(lines, list(range(10)), ncol=2);

The dynamics of *one-versus-rest* precision / recall scores on
$\mathcal{S}_\mathrm{valid}$. For binary classification:

$$ \begin{align}
\mathrm{Precision}
    &= \frac{\mathrm{TP}}{\mathrm{TP} + \mathrm{FP}}
        \approx \mathbb{P}(y = 1 \mid \hat{y} = 1)
    \,, \\
\mathrm{Recall}
    &= \frac{\mathrm{TP}}{\mathrm{TP} + \mathrm{FN}}
        \approx \mathbb{P}(\hat{y} = 1 \mid y = 1)
    \,.
\end{align}$$

In [ ]:
tp = scores.diagonal(axis1=-2, axis2=-1)
fp, fn = scores.sum(axis=-2) - tp, scores.sum(axis=-1) - tp

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 7))

lines = ax.plot(tp / (tp + fp), lw=2)
ax.set_title("Precision (ovr)")
ax.legend(lines, list(range(10)), ncol=2);

plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 7))

lines = ax.plot(tp / (tp + fn), lw=2)
ax.set_title("Recall (ovr)")
ax.legend(lines, list(range(10)), ncol=2)

plt.show()

The accuracy as a function of active learning iteration.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 7))

ax.plot(tp.sum(-1) / scores.sum((-2, -1)),
        label='Accuracy', lw=2)
ax.legend()

plt.show()

Volume of data used

In [ ]:
f"train : pool = {len(S_train)} : {len(S_pool)}"

Let the test set confusion matrix be the ultimate judge:

In [ ]:
score_matrix = evaluate(model, S_test, n_samples=51)

True positives, and false positives / negatives

In [ ]:
tp = score_matrix.diagonal(axis1=-2, axis2=-1)
fp, fn = score_matrix.sum(axis=-2) - tp, score_matrix.sum(axis=-1) - tp

accuracy

In [ ]:
f"(accuracy) {tp.sum() / score_matrix.sum():.2%}"

one-v-rest precision

In [ ]:
{l: f"{p:.2%}" for l, p in enumerate(tp / (tp + fp))}

ovr recall

In [ ]:
{l: f"{p:.2%}" for l, p in enumerate(tp / (tp + fn))}

<br>